In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install nfts

import numpy as np 
import pandas as pd 
import os
import sqlite3
import matplotlib.pyplot as plt
import nfts.dataset
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.special import zeta

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.8/500.8 kB 717.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for lru-dict: filename=lru_dict-1.1.7-cp37-cp37m-linux_x86_64.whl size=32518 sha256=076dbfce842df72e49e

In [2]:
os.listdir("/kaggle/input/ethereum-nfts")
DATASET_PATH = "/kaggle/input/ethereum-nfts/nfts.sqlite"
ds = nfts.dataset.FromSQLite(DATASET_PATH)

In [3]:
current_owners_df = ds.load_dataframe("current_owners")
current_owners_df

,nft_address,token_id,owner
0,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,0,0xb776cAb26B9e6Be821842DC0cc0e8217489a4581
1,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,1,0x8A73024B39A4477a5Dc43fD6360e446851AD1D28
2,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,10,0x5e5C817E9264B46cBBB980198684Ad9d14f3e0B4
3,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,11,0x8376f63c13b99D3eedfA51ddd77Ff375279B3Ba0
4,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,12,0xb5e34552F32BA9226C987769BF6555a538510BA8
...,...,...,...
7020945,0xff9315c2c4c0208Edb5152F4c4eBec75e74010c5,92,0x5d2e561a59B5AB7DA7ff9551502515CcDc9293d8
7020946,0xff9315c2c4c0208Edb5152F4c4eBec75e74010c5,98,0x052C65c1907eAD540C218648D4506760638D712b
7020947,0xffC6c59F34Cd9f8861012FDDd0c7F1323082Ab86,1,0x696B1c501c339f7057A0B58d730f2e5614f1CE7d
7020948,0xffCb352Fb3FdbEAab3F662378db28B8D151f210c,0,0x885fA6798114F28507977a07Ef122222B9dC74dD


In [4]:
current_owners_df.nunique()

nft_address       9292
token_id       1179659
owner           625354
dtype: int64

In [5]:
current_owners_grouped = current_owners_df.groupby("nft_address")
current_owners_grouped


In [6]:
groups = [name for name,unused_df in current_owners_grouped]
len(groups)

9292

In [7]:
nft_names = ds.load_dataframe("nfts")
nft_names.head(20)

,address,name,symbol
0,0xF4ee95274741437636e748DdAc70818B4ED7d043,The Doge Pound,DOGGY
1,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,BoredApeKennelClub,BAKC
2,0x7403AC30DE7309a0bF019cdA8EeC034a5507cbB3,Characters,CHAR
3,0x511372B44231a31527025a3D273C1dc0a83D77aF,Maps Restored,MAP
4,0x2E956Ed3D7337F4Ed4316A6e8F2EdF74BF84bb54,Neon Junkies,JUNKIES
5,0x03Ea00B0619e19759eE7ba33E8EB8E914fbF52Ea,pLOOT,pLOOT
6,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,CryptoKitties,CK
7,0xD81f156bBF7043a22d4cE97C0E8ca11d3f4FB3cC,Maps,MAP
8,0xD987C5800EF371844CEaC3D0Ee29E4ff29162d7C,SpaceLoot,SLOOT
9,0xdEcC60000ba66700a009b8F9F7D82676B5cfA88A,Settlements,STL


In [8]:
df_values = ds.load_dataframe("current_market_values")

In [9]:
# 1 WEI = 18 x 10^-16 USD
# 1 WEI = 10^-18 eth
df_values["eth"] = df_values["market_value"]* (10**(-18))

In [10]:
df_merge = df_values.merge(nft_names, left_on='nft_address', right_on='address')
df_merge

,nft_address,token_id,market_value,eth,address,name,symbol
0,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,0,3.000000e+16,0.03,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,zLoot,zLOOT
1,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,1,1.500000e+17,0.15,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,zLoot,zLOOT
2,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,10,1.500000e+17,0.15,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,zLoot,zLOOT
3,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,11,3.000000e+16,0.03,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,zLoot,zLOOT
4,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,12,9.000000e+16,0.09,0x00000000000b7F8E8E8Ad148f9d53303Bfe20796,zLoot,zLOOT
...,...,...,...,...,...,...,...
7020945,0xff9315c2c4c0208Edb5152F4c4eBec75e74010c5,92,0.000000e+00,0.00,0xff9315c2c4c0208Edb5152F4c4eBec75e74010c5,DeFiLABS,DLABS
7020946,0xff9315c2c4c0208Edb5152F4c4eBec75e74010c5,98,0.000000e+00,0.00,0xff9315c2c4c0208Edb5152F4c4eBec75e74010c5,DeFiLABS,DLABS
7020947,0xffC6c59F34Cd9f8861012FDDd0c7F1323082Ab86,1,0.000000e+00,0.00,0xffC6c59F34Cd9f8861012FDDd0c7F1323082Ab86,Hair Flip Puzzle Piece,GWPHFF
7020948,0xffCb352Fb3FdbEAab3F662378db28B8D151f210c,0,0.000000e+00,0.00,0xffCb352Fb3FdbEAab3F662378db28B8D151f210c,ATARINFT,ATRINFT


In [11]:
# df_merge_group = df_merge.groupby("nft_address")

In [12]:
#df_merge_group.get_group("0x00000000000b7F8E8E8Ad148f9d53303Bfe20796")

NameError: name 'df_merge_group' is not defined

In [13]:
df_merge_group = df_merge.groupby("nft_address")

In [29]:
df_merge_group = df_merge.groupby("nft_address")
df_search_20 = df_merge_group.max().sort_values(by=["eth"], ascending = False).head(20)

In [14]:
# df_max_eth = df_merge.groupby("nft_address")["eth"].max()
# df_maxeth_2 = df_max_eth.to_frame()

#df_masks = df_merge_group.agg({"eth":"max"})[["eth"]].reset_index()
#df_masks

#new_df = pd.merge(df_masks, df_merge, how='left', left_on=["nft_address", "eth"], right_on=["nft_address", "eth"])

In [15]:
#new_df.drop_duplicates(subset='nft_address', keep="first", inplace=True)

In [16]:
#new_df.sort_values(by=["eth"], ascending=False).head(20)

#first_20_df = new_df.sort_values(by=["eth"], ascending=False).head(20)

#first_20_projects = first_20_df["nft_address"]
#first_20_tokens = first_20_df["token_id"]

In [17]:
first_20_projects

159258    0xC36442b4a4522E871399CD717aBDD847Ab11FE88
75786     0x58A3c68e2D3aAf316239c003779F71aCb870Ee47
296149    0xd754937672300Ae6708a51229112dE4017810934
261191    0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270
473785    0xfD1539Ee9ccAD1aB5D9b4877097CfD8221815b47
108852    0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7
5984      0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a
284228    0xb9ed94c6d594b2517c4296e24A8c517FF133fb6d
285303    0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623
284154    0xb932a70A57673d89f4acfFBE830E8ed7f75Fb9e0
157001    0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D
48580     0x371020c4a69457935584F68000E4F883e4e8Ed98
295431    0xd4e4078ca3495DE5B1d4dB434BEbc5a986197782
157511    0xBd3531dA5CF5857e7CfAA92426877b022e612cf8
29540     0x1eFf5ed809C994eE2f500F076cEF22Ef3fd9c25D
86402     0x60E4d786628Fea6478F785A6d7e704777c86a7c6
245364    0xECf7EF42B57ee37A959BF507183C5dD6bf182081
56021     0x3D9b128cE781A4a9C1057f785D97131A6357ca38
55240     0x3AFF7B16489Fcc59483DE44e96Bd9Ec533

In [18]:
df_transfers = ds.load_dataframe("transfers")
df_transfers

,event_id,transaction_hash,block_number,nft_address,token_id,from_address,to_address,transaction_value,timestamp
0,cd816651-56b2-4ed9-887c-c83de732428d,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,6682934142305278177002530107138320246486863356...,0x25f1d709b329C7349b8209851E90eFa3a7f60178,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.000000e+00,1627776481
1,82cc5228-eb80-4e0d-9f6f-e644dec3ab06,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,1198507443711741845273832738859307575893198744...,0x0737E7162C88E9FBB963334e4Bfe6e97447fF811,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.000000e+00,1627776481
2,6e1f9cc4-d1df-4a6b-972d-a20765beb326,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,1541220344281436903042142177888467401468454835...,0xA0fEc186c4b3FAd39ca373dFdd52E79C2495F2c3,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.000000e+00,1627776481
3,43dc27a7-a72d-4894-809c-e868de05f7ee,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,7993840355029527745913866651030234226423689128...,0x90bE22069560A9bAd99543B4A922d1FE34Ae84E9,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.000000e+00,1627776481
4,47b7839b-9b87-442d-b2c1-9ebedcad8e06,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,1123719749142984581497870712442861055686734521...,0xA8E9C42Ebb915EeEC5080B7Bcc5D844475595091,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,0.000000e+00,1627776481
...,...,...,...,...,...,...,...,...,...
4514724,d45af108-4869-44bb-833d-2539faba0f1b,0xed4c976702534f735048e6ffca2a8704904f8aa96b77...,12545210,0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D,5175,0x1b826e30B83A5066474C7E3CA8D9b3880Ba97F09,0x519e19Eac9EAd404343c68224dD2A22C5c25F992,0.000000e+00,1622505430
4514725,73c81c59-c17a-4154-8bbc-5d72a27ee905,0x883257f2403851cf4f1f19a87728aafe269791fe0fcd...,12545211,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,2572313837129053907318259388395403352463424684...,0x327305A797d92a39cEe1a225D7E2A1cC42B1a8fA,0xDe747F2AF4627c1646C206eC7289597039ba319E,0.000000e+00,1622505446
4514726,27f9c904-06cf-4807-9011-d5ecae274249,0xd2643bdc7b6cd192fbdb721b62c8910fbf8753a17605...,12545211,0x629A673A8242c2AC4B7B8C5D8735fbeac21A6205,2576534060693998439509198862073888456960734364...,0x19804927f74C3c0899AFd49d56bff6ddAE6cF518,0x1b6A054a0850De061763B8C869daF53911e84C91,0.000000e+00,1622505446
4514727,89718578-d85b-493b-ab2f-8bd49187109c,0x3267f817704fe3fc328a348e169431c17ab07ecf757f...,12545211,0x3B3ee1931Dc30C1957379FAc9aba94D1C48a5405,43236,0xC5673A84A47aa7e7bcB66Fc64B157B8964D93a9b,0xcDA72070E455bb31C7690a170224Ce43623d0B6f,0.000000e+00,1622505446


In [35]:
df_transfer_merge = pd.merge(df_search_20, df_transfers, how="left", left_on=["nft_address", "token_id"], right_on=["nft_address", "token_id"])
df_transfer_merge

,nft_address,token_id,market_value,eth,address,name,symbol,event_id,transaction_hash,block_number,from_address,to_address,transaction_value,timestamp
0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,99999,1.890000e+22,18900.000000,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Uniswap V3 Positions NFT-V1,UNI-V3-POS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x58A3c68e2D3aAf316239c003779F71aCb870Ee47,9997961241216440618505325236136168061325754000714,1.500065e+22,15000.647964,0x58A3c68e2D3aAf316239c003779F71aCb870Ee47,Curve SynthSwap,CRV/SS,5f94457d-d405-4ffc-a62f-eec29689ec90,0x4f335f59c1011fc1cff763c52f970ade5b38cbd8e6a5...,12637516.0,0x21D6e3501Cf32b7dB4757a3931aFf19d515b68Ac,0x0000000000000000000000000000000000000000,0.000000e+00,1.623739e+09
2,0xd754937672300Ae6708a51229112dE4017810934,99,2.275000e+21,2275.000000,0xd754937672300Ae6708a51229112dE4017810934,Deafbeef,DEAFBEEF,2456852f-4b6c-4875-8b1f-fc358aa928dd,0x73543ee84f7c707ab43501d286a3e6115ebbab91f387...,12383611.0,0x5f298bd43168b7d3cBb71B1ea017ddbE250220ae,0xDe4ffb224371Ba792fdbD22691372a20F29F57e4,0.000000e+00,1.620342e+09
3,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,99001023,1.800000e+21,1800.000000,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,Art Blocks,BLOCKS,372a6937-b28a-4517-9b81-a6e656c6a3d5,0xacd03d86a162db53eb4c3ccc40890663893bac3b1e30...,13104752.0,0x0c16bE8F6Ace12e9B2160092bb89b29fD3117ef6,0x116928401Eb69Fa0c7b1671a2524D7f102353973,0.000000e+00,1.630031e+09
4,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,99001023,1.800000e+21,1800.000000,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,Art Blocks,BLOCKS,c4288bad-634d-4a85-b9f9-178fe3aa0f88,0x8b805d6a1bb08f99f4b12a53ee75200ebb505924f1a6...,13105009.0,0x116928401Eb69Fa0c7b1671a2524D7f102353973,0xe9659cA29deC45C493FABd388DD12Be3084b4c06,3.760000e+17,1.630034e+09
5,0xfD1539Ee9ccAD1aB5D9b4877097CfD8221815b47,999,1.000001e+21,1000.000990,0xfD1539Ee9ccAD1aB5D9b4877097CfD8221815b47,Mintable Governance NFT v2,MINT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7,9999,1.000000e+21,1000.000000,0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7,Meebits,⚇,a48a3644-fca8-4375-a3f1-7a82e8b5a602,0xa0bfa183417a180d1bd757ac628c7c9c700346f423f9...,12364518.0,0xf476Cd75BE8Fdd197AE0b466A2ec2ae44Da41897,0x0A383e69F43bD7CB5476fCedc1DF6ad9D4B09CF9,0.000000e+00,1.620088e+09
7,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,997,9.225000e+20,922.500000,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,Art Blocks,BLOCKS,380a6b30-b914-45f2-992a-1a09e26f1bbb,0xbe00f2f4c449075b5aa3c2911fda31be229ba778ebaa...,13281318.0,0x0eb9a7ff5cBf719251989CAf1599c1270Eafb531,0x19A0110537e4960c765D6f3Fea9445b022A287f0,0.000000e+00,1.632391e+09
8,0xb9ed94c6d594b2517c4296e24A8c517FF133fb6d,9,7.508888e+20,750.888800,0xb9ed94c6d594b2517c4296e24A8c517FF133fb6d,Hegic ETH ATM Calls Pool,ETHCALLSPOOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,9999,6.260000e+20,626.000000,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,BoredApeKennelClub,BAKC,53af5acb-cdc3-46ff-a874-393892b07240,0x58ffde4a81f7959320ab9b042337fdbe2ea5dd667516...,13094824.0,0xb0e984D4bf1830dcB6C8c4e0F81D0869D2f037b1,0x46Fd6B647Dc8C82aD8f6cf0CC6b22ACad3f6e39d,0.000000e+00,1.629898e+09


In [20]:
#mask1 = df_transfer_merge["nft_address"].isin(first_20_projects)
#mask2 = df_transfer_merge["token_id"].isin(first_20_tokens)

#df_masked = df_transfer_merge[mask1 & mask2]



In [38]:
df_transfer_merge.drop_duplicates(subset='nft_address', keep="first", inplace=True)
df_transfer_merge.reset_index(drop=True, inplace=True)

In [39]:
df_transfer_merge

,nft_address,token_id,market_value,eth,address,name,symbol,event_id,transaction_hash,block_number,from_address,to_address,transaction_value,timestamp
0,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,99999,1.890000e+22,18900.000000,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Uniswap V3 Positions NFT-V1,UNI-V3-POS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x58A3c68e2D3aAf316239c003779F71aCb870Ee47,9997961241216440618505325236136168061325754000714,1.500065e+22,15000.647964,0x58A3c68e2D3aAf316239c003779F71aCb870Ee47,Curve SynthSwap,CRV/SS,5f94457d-d405-4ffc-a62f-eec29689ec90,0x4f335f59c1011fc1cff763c52f970ade5b38cbd8e6a5...,12637516.0,0x21D6e3501Cf32b7dB4757a3931aFf19d515b68Ac,0x0000000000000000000000000000000000000000,0.000000e+00,1.623739e+09
2,0xd754937672300Ae6708a51229112dE4017810934,99,2.275000e+21,2275.000000,0xd754937672300Ae6708a51229112dE4017810934,Deafbeef,DEAFBEEF,2456852f-4b6c-4875-8b1f-fc358aa928dd,0x73543ee84f7c707ab43501d286a3e6115ebbab91f387...,12383611.0,0x5f298bd43168b7d3cBb71B1ea017ddbE250220ae,0xDe4ffb224371Ba792fdbD22691372a20F29F57e4,0.000000e+00,1.620342e+09
3,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,99001023,1.800000e+21,1800.000000,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,Art Blocks,BLOCKS,372a6937-b28a-4517-9b81-a6e656c6a3d5,0xacd03d86a162db53eb4c3ccc40890663893bac3b1e30...,13104752.0,0x0c16bE8F6Ace12e9B2160092bb89b29fD3117ef6,0x116928401Eb69Fa0c7b1671a2524D7f102353973,0.000000e+00,1.630031e+09
4,0xfD1539Ee9ccAD1aB5D9b4877097CfD8221815b47,999,1.000001e+21,1000.000990,0xfD1539Ee9ccAD1aB5D9b4877097CfD8221815b47,Mintable Governance NFT v2,MINT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7,9999,1.000000e+21,1000.000000,0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7,Meebits,⚇,a48a3644-fca8-4375-a3f1-7a82e8b5a602,0xa0bfa183417a180d1bd757ac628c7c9c700346f423f9...,12364518.0,0xf476Cd75BE8Fdd197AE0b466A2ec2ae44Da41897,0x0A383e69F43bD7CB5476fCedc1DF6ad9D4B09CF9,0.000000e+00,1.620088e+09
6,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,997,9.225000e+20,922.500000,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,Art Blocks,BLOCKS,380a6b30-b914-45f2-992a-1a09e26f1bbb,0xbe00f2f4c449075b5aa3c2911fda31be229ba778ebaa...,13281318.0,0x0eb9a7ff5cBf719251989CAf1599c1270Eafb531,0x19A0110537e4960c765D6f3Fea9445b022A287f0,0.000000e+00,1.632391e+09
7,0xb9ed94c6d594b2517c4296e24A8c517FF133fb6d,9,7.508888e+20,750.888800,0xb9ed94c6d594b2517c4296e24A8c517FF133fb6d,Hegic ETH ATM Calls Pool,ETHCALLSPOOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,9999,6.260000e+20,626.000000,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,BoredApeKennelClub,BAKC,53af5acb-cdc3-46ff-a874-393892b07240,0x58ffde4a81f7959320ab9b042337fdbe2ea5dd667516...,13094824.0,0xb0e984D4bf1830dcB6C8c4e0F81D0869D2f037b1,0x46Fd6B647Dc8C82aD8f6cf0CC6b22ACad3f6e39d,0.000000e+00,1.629898e+09
9,0xb932a70A57673d89f4acfFBE830E8ed7f75Fb9e0,9998,6.180000e+20,618.000000,0xb932a70A57673d89f4acfFBE830E8ed7f75Fb9e0,SuperRare,SUPR,cb25a931-23be-40d5-ba12-d94c8d559e0d,0xbdb9460998b3c03900d07e67b806449eb08d96d4b4d0...,13114682.0,0xc2FF6199CcB2FA07F867eD0e47E977e758f50d21,0x8c9F364bf7a56Ed058fc63Ef81c6Cf09c833e656,3.090000e+17,1.630164e+09


In [22]:
df_masked.sort_values(by=["eth"], ascending=False, inplace=True)

In [23]:
df_masked

,nft_address,eth,token_id,market_value,address,name,symbol,event_id,transaction_hash,block_number,from_address,to_address,transaction_value,timestamp
8682,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,18900.000000,64609,1.890000e+22,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Uniswap V3 Positions NFT-V1,UNI-V3-POS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4172,0x58A3c68e2D3aAf316239c003779F71aCb870Ee47,15000.647964,5999104731325796279969355981093260268301222429...,1.500065e+22,0x58A3c68e2D3aAf316239c003779F71aCb870Ee47,Curve SynthSwap,CRV/SS,be1fb3af-d63f-47ca-a711-fd507872ed3f,0xdf3e64e56cf81fb22b623c56fbb4e96388387f4946ea...,12489818.0,0x67e9a5894D2713553cd3cBC7D034Be9F1F830D3b,0x0000000000000000000000000000000000000000,0.000000e+00,1.621763e+09
11617,0xd754937672300Ae6708a51229112dE4017810934,2275.000000,130,2.275000e+21,0xd754937672300Ae6708a51229112dE4017810934,Deafbeef,DEAFBEEF,aa2e726b-5cd9-4759-8e93-366f7e603035,0x7b459405ae0a5538f26be77fb42b67f363a8bea660e1...,13051262.0,0xe1D29d0a39962a9a8d2A297ebe82e166F8b8EC18,0x924Ec2504CC71CdbD39011B417ac5a9D3d4dFF1b,2.275000e+21,1.629317e+09
10419,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,1800.000000,13000879,1.800000e+21,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,Art Blocks,BLOCKS,c077f0ec-bb09-498b-b2c8-854daf155924,0x71301ecceb449b3d15f1aea9dbd5f64917a2c3874337...,13107689.0,0xAfd37C11f9DFEe561db31D9bAb7cEc4812BBD241,0x75e1c2493136a2d62888615d53382AC3116861fE,1.800000e+21,1.630071e+09
12573,0xfD1539Ee9ccAD1aB5D9b4877097CfD8221815b47,1000.000990,504,1.000001e+21,0xfD1539Ee9ccAD1aB5D9b4877097CfD8221815b47,Mintable Governance NFT v2,MINT,c9c541d6-425c-40c1-836b-fc3ff875aadc,0x9fe475c625553d191833271ea1b9c1533a3c1c31f5fa...,12197784.0,0x02A522D98EC2D2c3bBe91AcC29ee7fD32ab880ab,0x6eD31BC364df5169D4A2C0f9e7c87Ec824C7E641,1.000001e+21,1.617867e+09
5736,0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7,1000.000000,17522,1.000000e+21,0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7,Meebits,⚇,b2dfe27e-f6f8-475b-897c-30ab1d027d01,0x99f1872f2a7bbd1e36910beed2d639378ebd3e747418...,12796259.0,0x12e5E3E7A1389feD96e1BE50aEd32316C027B18B,0x005E9EeD36BfEA0D05c0e8f36f32d4f4E08EFacd,1.000000e+21,1.625875e+09
344,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,922.500000,7583,9.225000e+20,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,Art Blocks,BLOCKS,ffbb18ec-19b6-4e4b-92e9-a6d9daf83b8b,0x958dbf5a6091fdcc9a5b46895e7cd8288cbf8e67846c...,13025462.0,0x6eF2376fA6e12Dabb3a3ED0Fb44E4ff29847Af68,0x9df947CD8a9810964473339c38a95201ECbE112d,0.000000e+00,1.628973e+09
10865,0xb9ed94c6d594b2517c4296e24A8c517FF133fb6d,750.888800,4,7.508888e+20,0xb9ed94c6d594b2517c4296e24A8c517FF133fb6d,Hegic ETH ATM Calls Pool,ETHCALLSPOOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10930,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,626.000000,3928,6.260000e+20,0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623,BoredApeKennelClub,BAKC,67bab8e7-18dc-4d8f-bde5-dcc72cc394b2,0xaf61094f73f33f8d38bf035a7c3b9598bb9884f90fe7...,13212487.0,0xea39c551834D07EE2EE87f1cEFF843c308e089AF,0xC310e760778ECBca4C65B6C559874757A4c4Ece0,5.000000e+15,1.631470e+09
10845,0xb932a70A57673d89f4acfFBE830E8ed7f75Fb9e0,618.000000,23154,6.180000e+20,0xb932a70A57673d89f4acfFBE830E8ed7f75Fb9e0,SuperRare,SUPR,9b87af97-3757-428b-b45c-e09565254210,0x6568317017d8722ecff2d2be77c2ddcc53e827541d86...,12205115.0,0x45Fa9AB78aE37fbB282F8069b49765ECAa04c0e7,0x0bD3D88aec644c098574C881c25e963d9f57CA57,6.180000e+20,1.617964e+09


In [24]:
df_masked["transaction_hash"]

8682                                                   NaN
4172     0xdf3e64e56cf81fb22b623c56fbb4e96388387f4946ea...
11617    0x7b459405ae0a5538f26be77fb42b67f363a8bea660e1...
10419    0x71301ecceb449b3d15f1aea9dbd5f64917a2c3874337...
12573    0x9fe475c625553d191833271ea1b9c1533a3c1c31f5fa...
5736     0x99f1872f2a7bbd1e36910beed2d639378ebd3e747418...
344      0x958dbf5a6091fdcc9a5b46895e7cd8288cbf8e67846c...
10865                                                  NaN
10930    0xaf61094f73f33f8d38bf035a7c3b9598bb9884f90fe7...
10845    0x6568317017d8722ecff2d2be77c2ddcc53e827541d86...
8488     0x46075de953b6438770f673a6ed19d8393230916cd152...
2687     0xeb3e7d6fc2c826f4828084af2c8720914f7f7aedfabf...
11559    0x60e3d99c34d98ab79482618bc7a58b475b47f536f334...
8558     0xd53daf8d912ec4a2984fcecbbc705d7f7c45a1317dc1...
1570     0xf49f32f6f6ed69454b57687f2aafa8dae7c39747eec1...
4625     0xcc23973667fcc69e4b1a5ff7680761e45c6376d6311e...
2949     0xc7bc734f230336d663e9b3fd2d25e83a7d4cfdb54560.

In [25]:
df_masked.reset_index(drop=True,inplace=True)

In [26]:
df_masked["transaction_hash"]

for i in range(20):
    print(df_masked["transaction_hash"][i])

nan
0xdf3e64e56cf81fb22b623c56fbb4e96388387f4946ea4acfc8aa00367b88e978
0x7b459405ae0a5538f26be77fb42b67f363a8bea660e1f0ce2042cf8fc6da4fbf
0x71301ecceb449b3d15f1aea9dbd5f64917a2c3874337e305e6e5cb692914b843
0x9fe475c625553d191833271ea1b9c1533a3c1c31f5fa1cc2a812b0bb0cdf2022
0x99f1872f2a7bbd1e36910beed2d639378ebd3e747418c184b2159da7e25113e5
0x958dbf5a6091fdcc9a5b46895e7cd8288cbf8e67846cbe53c4f6fba8a26b6cc1
nan
0xaf61094f73f33f8d38bf035a7c3b9598bb9884f90fe77a683c9692c731a67e6d
0x6568317017d8722ecff2d2be77c2ddcc53e827541d86eb4598243ae8f9b0bb07
0x46075de953b6438770f673a6ed19d8393230916cd15229b3d1a1a4750e985cdb
0xeb3e7d6fc2c826f4828084af2c8720914f7f7aedfabf46b99dbee871a3dc3e50
0x60e3d99c34d98ab79482618bc7a58b475b47f536f3348422a4c3126b70ca72cd
0xd53daf8d912ec4a2984fcecbbc705d7f7c45a1317dc1f9f596ba2f751d2abaef
0xf49f32f6f6ed69454b57687f2aafa8dae7c39747eec1b6081b7dec647b1708be
0xcc23973667fcc69e4b1a5ff7680761e45c6376d6311e72da2367df1fd755af1d
0xc7bc734f230336d663e9b3fd2d25e83a7d4cfdb54560cc05dccf

Nisan eylül arası alınıp satılmış en pahalı 20 NFT'yi buluyoruz. (txn hash üzerinden)

1- https://etherscan.io/nft/0xd754937672300ae6708a51229112de4017810934/130

2- https://etherscan.io/nft/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/13000879

3- https://opensea.io/assets/ethereum/0xfd1539ee9ccad1ab5d9b4877097cfd8221815b47/504

4- https://opensea.io/assets/ethereum/0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7/17522

5- https://opensea.io/assets/ethereum/0xba30e5f9bb24caa003e9f2f0497ad287fdf95623/3928 (?)


-Sıralarını bilmiyoruz-

https://etherscan.io/nft/0x059edd72cd353df5106d2b9cc5ab83a52287ac3a/7583
https://opensea.io/assets/ethereum/0x1eff5ed809c994ee2f500f076cef22ef3fd9c25d/103
https://opensea.io/assets/ethereum/0x371020c4a69457935584f68000e4f883e4e8ed98/40


In [27]:
df_merge_unique

NameError: name 'df_merge_unique' is not defined

In [ ]:
current_owners_grouped = current_owners_df.groupby("nft_address")

In [ ]:
df_values

In [ ]:
df_deneme = df_values.sort_values(by=['market_value'], ascending=False).iloc[0]

Nft address: Nft projesinin adresi, ör: Bored Aped Yacht Club

Token id: Nft'nin unique id'si

Market Value: WEI cinsinden fiyatı

USD: WEI -> USD

In [ ]:
current_owners_grouped = current_owners_df.groupby("nft_address")

In [ ]:
current_owners_grouped